In [6]:
import numpy as np
import pandas as pd
import joblib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


## División de datos

Al igual que con train_data, debido al tamaño del dataframe de test_data, es necesario dividirlo en lotes para trabajar con ellos individualmente y no cargar la memoria.
<br><br> Vamos a usar exactamente el mismo código que usamos para train_data 

In [15]:
# Cargamos el archivo de label completo
file_name = f'../data/test_data.csv'
final_label = pd.read_csv(file_name, encoding='ISO-8859-1')

In [17]:
# Dividimos el archivo en chunks
chunk_size = 500000

current_chunk = []
current_chunk_size = 0
file_index = 1
last_customer_id = None

# Función para guardar el chunk actual en un CSV
def save_chunk(chunk, file_index):
    df_chunk = pd.DataFrame(chunk)
    df_chunk.to_csv(f'test_data_part_{file_index}.csv', index=False)
    print(f'Archivo test_data_part_{file_index}.csv guardado con {len(df_chunk)} filas.')


for i, row in final_label.iterrows():
    customer_id = row['ID']

    # Si hemos alcanzado el tamaño máximo del chunk y el nuevo customer_id es diferente al último en el chunk
    if current_chunk_size >= chunk_size and customer_id != last_customer_id:
        # Guardar el CSV actual
        save_chunk(current_chunk, file_index) 
        file_index += 1
        
        # Reiniciar el chunk
        current_chunk = []  
        current_chunk_size = 0

    # Añadir la fila actual al chunk
    current_chunk.append(row)
    current_chunk_size += 1
    last_customer_id = customer_id

# Guardar el último chunk si no está vacío
if current_chunk:
    save_chunk(current_chunk, file_index)


Archivo test_data_part_1.csv guardado con 500009 filas.
Archivo test_data_part_2.csv guardado con 500002 filas.
Archivo test_data_part_3.csv guardado con 500011 filas.
Archivo test_data_part_4.csv guardado con 500005 filas.
Archivo test_data_part_5.csv guardado con 212637 filas.


In [18]:
for i in range(1, 6):
    file_name = f'test_data_part_{i}.csv'
    globals()[f'df_data_part_{i}'] = pd.read_csv(file_name, encoding='ISO-8859-1')
    print(f'Archivo {file_name} cargado en df_data_part_{i}')

Archivo test_data_part_1.csv cargado en df_data_part_1
Archivo test_data_part_2.csv cargado en df_data_part_2
Archivo test_data_part_3.csv cargado en df_data_part_3
Archivo test_data_part_4.csv cargado en df_data_part_4
Archivo test_data_part_5.csv cargado en df_data_part_5


# Preprocesado  


Este apartado consiste en el mismo preprocesado que usamos sobre train_data ya que los datos de test tienen que tener la misma estructura que los datos con los que se entrenó el modelo (Nº de columnas, tipo de datos de columnas... ) o dará un problema al realizar la predicción

### Comprobar columnas con valores nulos

In [19]:
def null_percentage(df, percentage):
    null_ratio = {}
    for col in df.columns:
        ratio = df[col].isna().sum() / len(df) * 100
        if ratio > percentage:
            null_ratio[col] = ratio
           
    return null_ratio

Me quedo solo con las columnas con menos de 50% de valores nulos

In [20]:
columns_with_high_nulls = []
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    null_columns = set(null_percentage(data_frame, 50).keys())
    
    # Si es el primer DataFrame, inicializar el conjunto con sus columnas
    if i == 1:
        columns_with_high_nulls = null_columns
    else:
        # Mantener solo las columnas que están en todos los DataFrames
        columns_with_high_nulls = columns_with_high_nulls.intersection(null_columns)

columns_with_high_nulls = list(columns_with_high_nulls)

print(f"Columnas con más del 50% de valores nulos en todos los CSVs: {columns_with_high_nulls}")

Columnas con más del 50% de valores nulos en todos los CSVs: ['Infraction_JVWF', 'Infraction_ZVHJ', 'Base_8318', 'Infraction_SBF', 'Expenditure_KMW', 'Infraction_MZI', 'Infraction_NCB', 'Infraction_IRKE', 'Infraction_CLLY', 'Infraction_ADWZ', 'Infraction_GEL', 'Risk_4561', 'Infraction_WEG', 'Infraction_SVKR', 'Infraction_APIU', 'Infraction_ZTLC', 'Infraction_MAN', 'Infraction_GWL', 'Base_64022', 'Infraction_FUSM', 'Infraction_HPS', 'Base_3958', 'Infraction_WLMI', 'Infraction_QGC', 'Infraction_WWLN', 'Infraction_HPLO', 'Base_8379', 'Infraction_EBA', 'Infraction_ANHZ', 'Risk_5797']


Eliminamos las columnas con más de 50% de valores nulos en todos los csv's

In [21]:
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame.drop(columns=columns_with_high_nulls, inplace=True)
    

### Comprobar si hay filas que tengan todas las columnas a null


In [22]:
# Verificar si hay filas completamente nulas
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    null_rows = data_frame[data_frame.isnull().all(axis=1)]
    if not null_rows.empty:
        print(f'Filas completamente nulas en df_data_part_{i}:')
        print(null_rows)
    else:
        print(f'No hay filas completamente nulas en df_data_part_{i}.')

No hay filas completamente nulas en df_data_part_1.
No hay filas completamente nulas en df_data_part_2.
No hay filas completamente nulas en df_data_part_3.
No hay filas completamente nulas en df_data_part_4.
No hay filas completamente nulas en df_data_part_5.


### Comprobar filas duplicadas

In [23]:
# Verificar si hay filas duplicadas en cada DataFrame
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    duplicate_rows = data_frame[data_frame.duplicated()]
    if not duplicate_rows.empty:
        print(f'Filas duplicadas en df_data_part_{i}:')
        print(duplicate_rows)
    else:
        print(f'No hay filas duplicadas en df_data_part_{i}.')

No hay filas duplicadas en df_data_part_1.
No hay filas duplicadas en df_data_part_2.
No hay filas duplicadas en df_data_part_3.
No hay filas duplicadas en df_data_part_4.
No hay filas duplicadas en df_data_part_5.


### Comprobar varianza 0

In [24]:
# Verificar si hay columnas con baja varianza
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    low_variance_cols = []
    for col in data_frame.columns:
        if data_frame[col].nunique() == 1:
            low_variance_cols.append(col)
    if low_variance_cols:
        print(f'Columnas con baja varianza en df_data_part_{i}: {low_variance_cols}')
    else:
        print(f'No hay columnas con baja varianza en df_data_part_{i}.')

No hay columnas con baja varianza en df_data_part_1.
No hay columnas con baja varianza en df_data_part_2.
No hay columnas con baja varianza en df_data_part_3.
No hay columnas con baja varianza en df_data_part_4.
No hay columnas con baja varianza en df_data_part_5.


### Análisis de tipo de variable

In [26]:
# Agrupar columnas por tipo de dato
data_types = {}
# No hace falta comprobar todos los df ya que tienen las mismas columnas
data_frame = df_data_part_1
for col in data_frame.columns:
    data_type = str(data_frame[col].dtype)
    if data_type not in data_types:
        data_types[data_type] = []
    data_types[data_type].append(col)

for data_type, columns in data_types.items():
    print(f'{data_type}: {columns}')

object: ['ID', 'Expenditure_AHF', 'Infraction_YFSG', 'Infraction_DQLY', 'Infraction_CLH', 'Base_67254', 'Infraction_TEN']
float64: ['Payment_6804', 'Infraction_CGP', 'Base_7744', 'Base_80863', 'Risk_1930', 'Expenditure_JIG', 'Infraction_SNZ', 'Base_02683', 'Infraction_ZWWJ', 'Infraction_QJJF', 'Base_76065', 'Infraction_EJZ', 'Base_6872', 'Risk_0322', 'Infraction_FMXQ', 'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598', 'Base_6187', 'Infraction_ZTNC', 'Base_85131', 'Risk_9995', 'Infraction_AYWV', 'Payment_22507', 'Base_9516', 'Expenditure_YTR', 'Base_36384', 'Expenditure_FIP', 'Infraction_PAS', 'Risk_0003', 'Expenditure_HMO', 'Base_24406', 'Expenditure_LMSR', 'Infraction_BSU', 'Base_14808', 'Risk_8065', 'Infraction_ZYW', 'Base_1039', 'Infraction_HSSU', 'Infraction_EHZP', 'Infraction_TBP', 'Base_0580', 'Expenditure_RGD', 'Infraction_PBC', 'Infraction_AQO', 'Base_0229', 'Base_69608', 'Base_91828', 'Base_6852', 'Expenditure_IDZ', 'Risk_1475', 'Expenditure_BWX', 'Base_8511', 'I

#### Transformación ID to_numeric

In [27]:
# Pasamos ID a número en todos los dataframes para facilitar su procesado
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['ID'] = pd.to_numeric(data_frame['ID'], errors='coerce')

#### Transformación Expenditure_AHF to_datetime

In [29]:
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Expenditure_AHF'] = pd.to_datetime(data_frame['Expenditure_AHF'], errors='coerce')
    data_frame['Expenditure_AHF_year'] = data_frame['Expenditure_AHF'].dt.year
    data_frame['Expenditure_AHF_month'] = data_frame['Expenditure_AHF'].dt.month
    data_frame['Expenditure_AHF_day'] = data_frame['Expenditure_AHF'].dt.day
    data_frame.drop(columns=['Expenditure_AHF'], inplace=True)


#### Transformación Infraction_YFSG encoding

In [31]:
# Ver los valores de la columna Infraction_YFSG en todos los DataFrames y hacer una intersección
unique_values = set()
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    unique_values_i = set(data_frame['Infraction_YFSG'].unique())
    if not unique_values:
        unique_values = unique_values_i
    else:
        unique_values = unique_values.intersection(unique_values_i)

print(unique_values)

{'XL', 'CL', 'XZ', 'CR', 'CO', 'XM'}


In [32]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Infraction_YFSG_encoded'] = label_encoder.fit_transform(data_frame['Infraction_YFSG'])
    data_frame.drop(columns=['Infraction_YFSG'], inplace=True)


#### Transformación Infraction_DQLY

In [33]:
# Ver los valores de la columna Infraction_YFSG en todos los DataFrames y hacer una intersección
unique_values = set()
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    unique_values_i = set(data_frame['Infraction_DQLY'].unique())
    if not unique_values:
        unique_values = unique_values_i
    else:
        unique_values = unique_values.intersection(unique_values_i)

print(unique_values)

{'-1', nan, 'U', 'R', 'O'}


In [34]:
# Análisis de porcentaje de NaNs en cada DataFrame en la columna Infraction_DQLY
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    print(f'Porcentaje de NaNs en df_data_part_{i}: {data_frame["Infraction_DQLY"].isna().mean() * 100:.2f}%')

Porcentaje de NaNs en df_data_part_1: 3.90%
Porcentaje de NaNs en df_data_part_2: 3.85%
Porcentaje de NaNs en df_data_part_3: 4.00%
Porcentaje de NaNs en df_data_part_4: 3.92%
Porcentaje de NaNs en df_data_part_5: 3.96%


Antes de codificarlo necesitamos tratar los NaN

In [35]:
# Como el porcentaje de NaNs es muy bajo (aprox. 4%), rellenamos los NaNs con el valor más común
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    most_common_value = data_frame['Infraction_DQLY'].mode()[0]
    data_frame['Infraction_DQLY'].fillna(most_common_value, inplace=True)



C:\Users\eleni\AppData\Local\Temp\ipykernel_30248\3431230910.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_frame['Infraction_DQLY'].fillna(most_common_value, inplace=True)


##### Hacemos la codificación de la columna

In [36]:
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Infraction_DQLY_encoded'] = label_encoder.fit_transform(data_frame['Infraction_DQLY'])
    data_frame.drop(columns=['Infraction_DQLY'], inplace=True)


### Las siguientes columnas comparten etiquetas

In [37]:
ordinal_columns = ['Infraction_CLH', 'Base_67254', 'Infraction_TEN']

for col in ordinal_columns:
    print(f'{col}: {df_data_part_1[col].unique()}')

Infraction_CLH: ['very_high' nan 'moderate_low' 'moderate' 'high' 'moderate_high' 'low'
 'very_low']
Base_67254: ['moderate_low' 'low' 'high' 'moderate' 'moderate_high' 'very_high'
 'extremely_high' nan]
Infraction_TEN: ['moderate_high' 'extremely_low' nan 'moderate_low' 'very_high' 'moderate'
 'high' 'low']


In [38]:
frequency_map = {
    'extremely_low': 0,
    'very_low': 1,
    'moderate_low': 2,
    'low': 3,
    'moderate': 4,
    'high':5,
    'moderate_high': 6,
    'very_high': 7,
    'extremely_high': 8
}


for i in range(1, 6):
    for col in ordinal_columns:
        data_frame = globals()[f'df_data_part_{i}']
        data_frame[f'{col}_encoded'] = data_frame[col].map(frequency_map)
        data_frame.drop(columns=[col], inplace=True)

In [39]:
ordinal_columns = ['Infraction_CLH_encoded', 'Base_67254_encoded', 'Infraction_TEN_encoded']

for col in ordinal_columns:
    print(f'{col}: {df_data_part_1[col].unique()}')

Infraction_CLH_encoded: [ 7. nan  2.  4.  5.  6.  3.  1.]
Base_67254_encoded: [ 2.  3.  5.  4.  6.  7.  8. nan]
Infraction_TEN_encoded: [ 6.  0. nan  2.  7.  4.  5.  3.]


In [40]:
# Voy a ver cuántos valores NaN hay en cada columna de las ordinal_columns en la intersection de los DataFrames como porcentaje
for col in ordinal_columns:
    nan_count = 0
    total = 0
    for i in range(1, 6):
        data_frame = globals()[f'df_data_part_{i}']
        total += data_frame[col].shape[0]
        nan_count += data_frame[col].isna().sum()
    print(f'{col}: {nan_count/total*100:.2f}%')

Infraction_CLH_encoded: 3.91%
Base_67254_encoded: 0.03%
Infraction_TEN_encoded: 3.18%


Es un porcentaje muy bajo así que podemos rellenarlos sin problema. En este caso usaremos la moda

In [41]:
# Quiero saber el valor que más se repite en cada columna ordinal
for col in ordinal_columns:
    most_common_values = []
    for i in range(1, 6):
        data_frame = globals()[f'df_data_part_{i}']
        most_common_value = data_frame[col].mode()[0]
        most_common_values.append(most_common_value)
    print(f'{col}: {most_common_values}')
    

Infraction_CLH_encoded: [np.float64(7.0), np.float64(7.0), np.float64(7.0), np.float64(7.0), np.float64(7.0)]
Base_67254_encoded: [np.float64(2.0), np.float64(2.0), np.float64(2.0), np.float64(2.0), np.float64(2.0)]
Infraction_TEN_encoded: [np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]


In [42]:
# Rellenar los valores NaN con el valor más común en cada columna ordinal.
for col in ordinal_columns:
    for i in range(1, 6):
        data_frame = globals()[f'df_data_part_{i}']
        most_common_value = data_frame[col].mode()[0]
        data_frame[col] = data_frame[col].fillna(most_common_value)


### Comprobación columnas duplicadas

In [45]:
# Ya hemos visto que las categóricas no están dulpicadas entre sí así que solo analizamos si las columnas numéricas están duplicadas
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    duplicate_columns = data_frame.columns[data_frame.columns.duplicated()]
    if not duplicate_columns.empty:
        print(f'Columnas duplicadas en df_data_part_{i}: {duplicate_columns}')
    else:
        print(f'No hay columnas duplicadas en df_data_part_{i}.')


No hay columnas duplicadas en df_data_part_1.
No hay columnas duplicadas en df_data_part_2.
No hay columnas duplicadas en df_data_part_3.
No hay columnas duplicadas en df_data_part_4.
No hay columnas duplicadas en df_data_part_5.


### Comprobación NaNs

Vamos a analizar las filas para eliminar aquellas que tengan NaN en más del 30% de las columnas si estas componen una pequeña poción del dataframe y rellenar estos valores no tendría sentido ya que la fila perdería valor para el posterior estudio ya que sería completamente artificial

In [46]:
# Quiero saber cuántas filas tienen más del 30% de las columnas nulas en porcentaje sobre el total de todos los DataFrames
rows_to_drop = []
total_rows = 0
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    total_rows += data_frame.shape[0]
    null_percentage = data_frame.isnull().mean(axis=1) * 100
    rows_to_drop.extend(null_percentage[null_percentage > 30].index)
rows_to_drop = set(rows_to_drop)
print(f'Número de filas a eliminar: {len(rows_to_drop)/total_rows*100:.2f}%')


Número de filas a eliminar: 0.26%


In [47]:
# Eliminos las filas con más del 30% de valores nulos
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    # Como no me deja usar los valores de rows_to_drop como índices, lo hago de la siguiente manera
    data_frame.drop(data_frame.index.intersection(rows_to_drop), inplace=True)


Ahora vamos a hacer un análisis de las columnas que nos quedan

In [48]:
# Voy a ver cuántos valores NaN hay en cada columna en la intersection de los DataFrames.
columns_percentage = {}
for col in df_data_part_1.columns:
    nan_count = 0
    total_rows = 0
    for i in range(1, 6):
        data_frame = globals()[f'df_data_part_{i}']
        total_rows += data_frame.shape[0]
        nan_count += data_frame[col].isna().sum()
    if nan_count > 0:
        columns_percentage[col] = nan_count/total_rows * 100    
        print(f'{col}: {nan_count/total_rows * 100:.5f}%')

Payment_6804: 0.57987%
Base_80863: 0.00380%
Expenditure_JIG: 18.46015%
Infraction_SNZ: 0.00380%
Base_02683: 0.00380%
Infraction_ZWWJ: 29.71958%
Infraction_QJJF: 4.70174%
Infraction_EJZ: 0.00384%
Infraction_FMXQ: 21.72539%
Infraction_TLPJ: 12.83084%
Base_1165: 0.00018%
Base_6187: 0.37463%
Infraction_AYWV: 0.28399%
Payment_22507: 5.17378%
Infraction_PAS: 0.00380%
Expenditure_HMO: 18.46015%
Infraction_BSU: 3.13765%
Base_14808: 0.74555%
Infraction_HSSU: 1.68441%
Infraction_TBP: 10.75337%
Base_0580: 0.05204%
Infraction_PBC: 13.60488%
Base_0229: 0.00380%
Base_91828: 0.00380%
Base_6852: 0.00380%
Infraction_JYZB: 3.26058%
Base_22178: 0.00380%
Infraction_ZTYG: 1.44665%
Infraction_EYU: 0.18233%
Infraction_QKZN: 0.27311%
Infraction_JBR: 45.37687%
Base_66195: 0.05204%
Base_36516: 0.00380%
Infraction_RXQH: 4.70174%
Infraction_HFU: 1.10742%
Infraction_VTR: 0.27311%
Base_7331: 0.00380%
Infraction_XWX: 0.23186%
Risk_4553: 0.00146%
Infraction_VHU: 3.26058%
Infraction_GSS: 0.28399%
Base_8730: 0.00380%
R

Vamos a rellenar aquellas que tengan menos del 6% de NaNs y a eliminar los que tengan más de ese porcentaje , rellenar estos últimos no tendría sentido

In [49]:
# Eliminamos las columnas con más de 6% de valores nulos en la intersección de los DataFrames
for col in columns_percentage.keys():
    if columns_percentage[col] > 6:
        for i in range(1, 6):
            data_frame = globals()[f'df_data_part_{i}']
            data_frame.drop(columns=[col], inplace=True)    

In [50]:
# Vamos a rellenar los NaNs de las columnas restantes de la siguiente manera:
# Si la columna es categórica, rellena con la moda (no hace falta tenerlo en cuenta ahora ya que lo hemos hecho manualmnete anteriormente)
# Si la columna es numérica:
#           - con la media si la desviación estándar es menor a 1
#           - con la mediana si la desviación estándar es mayor a 1
#           - con la moda si la desviación estándar es 0

for col in df_data_part_1.columns:
    for i in range(1, 6):
        data_frame = globals()[f'df_data_part_{i}']
        if data_frame[col].isna().sum() != 0:
            mean = data_frame[col].mean()
            median = data_frame[col].median()
            mode = data_frame[col].mode()[0]
            std = data_frame[col].std()
            if std == 0:
                data_frame[col] = data_frame[col].fillna(mode)
            elif std < 1:
                data_frame[col] = data_frame[col].fillna(mean)
            else:
                data_frame[col] = data_frame[col].fillna(median)


### Tratar valores infinitos

In [52]:
# Imprimir los valores infinitos para tratarlos
from numpy import inf
for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    infinite_values = data_frame[data_frame == np.inf].sum().sum()
    print(f'Valores infinitos en df_data_part_{i}: {infinite_values}')

Valores infinitos en df_data_part_1: 0.0
Valores infinitos en df_data_part_2: 0.0
Valores infinitos en df_data_part_3: 0.0
Valores infinitos en df_data_part_4: 0.0
Valores infinitos en df_data_part_5: 0.0


#### Escalado

In [53]:
from sklearn.preprocessing import StandardScaler

for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']
    
    X = data_frame  
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    
    globals()[f'df_scaled_part_{i}'] = X_scaled



#### Baja varianza

In [54]:
# Voy a hacer una selección de características a eliminar con el método de la varianza (threshold=0.01)
from sklearn.feature_selection import VarianceThreshold
columnas_baja_varianza_final = None

for i in range(1, 6):
    data_frame = globals()[f'df_data_part_{i}']

    X = data_frame

    selector = VarianceThreshold(threshold=0.01)
    selector.fit(X)
    columnas_baja_varianza = X.columns[~selector.get_support()].tolist()
    if i == 1:
        columnas_baja_varianza_final = set(columnas_baja_varianza)
    else:
        columnas_baja_varianza_final.intersection_update(columnas_baja_varianza)

print(f'Columnas a eliminar por baja varianza en todas las particiones: {columnas_baja_varianza_final}')

Columnas a eliminar por baja varianza en todas las particiones: {'Risk_6197', 'Risk_5270', 'Infraction_KSBR', 'Risk_8742', 'Base_23737', 'Base_5441', 'Infraction_ZRH', 'Base_7331', 'Risk_9247', 'Risk_3506', 'Infraction_PTY', 'Risk_6178', 'Expenditure_GCAO', 'Risk_4160'}


In [55]:
for i in range(1,6):
    data_frame = globals()[f'df_scaled_part_{i}']
    data_frame.drop(columns=columnas_baja_varianza_final, inplace=True)

## Selección de características 

In [67]:
selected_features = list(pd.read_pickle('selected_features_lists/selected_features_CI.pkl'))

In [70]:
for i in range(1,6):
    data_frame = globals()[f'df_scaled_part_{i}']
    globals()[f'df_final_test_{i}'] =  data_frame[['ID'] + selected_features ]


## Guardado de dataframes preprocesados


In [72]:
for i in range(1, 6):
    globals()[f'df_final_test_{i}'].to_csv(f'test_data_part_{i}.csv', index=False)

# Mejor modelo: red de neuronas 

Para entrenar el modelo, cargamos los datos preprocesados de train y ajustamos los hiperparámetros de la red de neuronal para conseguir los mejores resultados

### Cargado de train dataframes

In [63]:
for i in range(1, 8):
    file_name_train = f'../data/dataFrame_final/final_train/df_final_train_{i}.csv'
    globals()[f'df_train_{i}'] = pd.read_csv(file_name_train, encoding='ISO-8859-1')
    print(f'TRAIN_{i}: Archivo df_train_{i} cargado')


TRAIN_1: Archivo df_train_1 cargado
TRAIN_2: Archivo df_train_2 cargado
TRAIN_3: Archivo df_train_3 cargado
TRAIN_4: Archivo df_train_4 cargado
TRAIN_5: Archivo df_train_5 cargado
TRAIN_6: Archivo df_train_6 cargado
TRAIN_7: Archivo df_train_7 cargado


### Entrenamiento

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:

def create_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(255, activation='relu'),
        Dropout( 0.31666966256912804),
        Dense(124, activation='relu'),
        Dropout(0.3008937986275955),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  
    ])
    return model

input_shape = globals()['df_train_1'].shape[1] - 1  
model = create_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

epochs = 50
batch_size = 512

early_stopping = EarlyStopping(
    monitor='loss', 
    patience=3,          
    restore_best_weights=True  
)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    for i in range(1, 8):  
        data_train = globals()[f'df_train_{i}']
        X_train = data_train.drop(columns=['label']).values
        y_train = data_train['label'].values

        model.fit(X_train, y_train, epochs=1, batch_size=batch_size, callbacks=[early_stopping], verbose=1)



# Predicción con el modelo ajustado

### Cargado de datos

In [7]:
model = joblib.load("../data/models/Neural_network.joblib")

In [74]:
for i in range(1,6):
    file_name_train = f'../test_data/test_data_part_{i}.csv'
    globals()[f'df_data_part_{i}'] = pd.read_csv(file_name_train, encoding='ISO-8859-1')
    print(f'Archivo test_data_part_{i}.csv cargado')

Archivo test_data_part_1.csv cargado
Archivo test_data_part_2.csv cargado
Archivo test_data_part_3.csv cargado
Archivo test_data_part_4.csv cargado
Archivo test_data_part_5.csv cargado


### Predicción

In [75]:
for i in range(1,6):
    data = globals()[f'df_data_part_{i}']
    X = data.values
    raw_predictions = model.predict(X)
    predictions = (raw_predictions > 0.5).astype(int)
    globals()[f'df_data_part_{i}']['label'] = predictions

15445/15445 ━━━━━━━━━━━━━━━━━━━━ 9s 560us/step
15444/15444 ━━━━━━━━━━━━━━━━━━━━ 7s 469us/step
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 7s 484us/step
15444/15444 ━━━━━━━━━━━━━━━━━━━━ 7s 482us/step
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 3s 490us/step


In [83]:
total_clients = set()  

for i in range(1, 6):
    data = globals()[f'df_data_part_{i}']
    total_clients.update(data['ID'].unique())  

print(f'Número total de clientes: {len(total_clients)}')


Número total de clientes: 183459


El dataframe test_labels debe de contener una sola fila por cliente asociado a una columna 'label' que determina si el cliente es fraudulento o no. Ahora sabemos que todo el dataframe de test contiene la información de 183.459 clientes

Vamos a fusionar las predicciones para label de cada transacción de un único cliente siguiendo la siguiente lógica: 
- Si una sola transacción de un cliente se considera fraudulenta, label será 1
- Si todas las transacciones son 0, label será 0

In [84]:
import pandas as pd

final_label = pd.DataFrame()

for i in range(1, 6):
    
    data = globals()[f'df_data_part_{i}']
    
    
    data = data[['ID', 'label']]
    
    # Usamos max() para que si alguna transacción tiene 'label' 1, se asigne 1 al cliente
    data_grouped = data.groupby('ID', as_index=False).agg({'label': 'max'})
    
    final_label = pd.concat([final_label, data_grouped], ignore_index=True)

# Agrupamos nuevamente por 'ID' en caso de que haya clientes duplicados entre partes y aplicamos la misma lógica
# final_label = final_label.groupby('ID', as_index=False).agg({'label': 'max'})


In [85]:
final_label.shape

(183459, 2)

In [89]:
final_label.to_csv('test_labels.csv', index=False)